# Importing MAAP collection as UMM_JSON for grouping collection that would be used to create examples

In [ ]:
import json
from urllib.request import urlopen
import pandas as pd
maap = 'https://cmr.ops.maap-project.org/search/collections.umm_json?page_size=1000'
response = urlopen(maap)
maap_data = json.load(response)

## Display the json data to understand structure

In [ ]:
import pprint
pp=pprint.PrettyPrinter(depth=5,compact=True)
pp.pprint(maap_data)

Removed output here as it was pretty long.

<br>
<br>

#### Based on structure we saw above, we determine that our needed variables are nested under: 
##### 1. items>umm>AdditionalAttributes> Spatial Resolution (2),DataFormat (4),AcquisitionType (8)
##### 2. items[0].umm.ScienceKeywords[0].Category, items[0].umm.ScienceKeywords[0].Topic
##### 3. items[0].umm.ShortName
##### 4. items[0].umm.LocationKeywords[0], items[0].umm.LocationKeywords[1]
##### 5. items[0].umm.Projects[0].ShortName, x.items[0].umm.Projects[0].LongName
##### 6. items[0].umm.Platforms[0].ShortName, items[0].umm.Platforms[0].LongName,items[0].umm.Platforms[0].Instruments[0].ShortName, items[0].umm.Platforms[0].Instruments[0].LongName

<br>
<br>

Using json_normalize to convert the entire json data to dataframe:

In [13]:
maap_normalized = pd.json_normalize(maap_data['items'])

In [11]:
maap_normalized

,meta.revision-id,meta.deleted,meta.format,meta.provider-id,meta.user-id,meta.has-formats,meta.has-spatial-subsetting,meta.native-id,meta.has-transforms,meta.has-variables,...,umm.UseConstraints.LicenseText,umm.AccessConstraints.Description,umm.AccessConstraints.Value,umm.UseConstraints.Description,umm.PublicationReferences,umm.SpatialExtent.HorizontalSpatialDomain.ResolutionAndCoordinateSystem.HorizontalDataResolution.GenericResolutions,umm.TemporalKeywords,umm.ContactGroups,umm.SpatialInformation.SpatialCoverageType,umm.TilingIdentificationSystems
0,13,False,application/echo10+xml,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_234,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,False,application/vnd.nasa.cmr.umm+json,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_394,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,False,application/echo10+xml,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_298,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17,False,application/echo10+xml,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_285,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,False,application/echo10+xml,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_231,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1,False,application/vnd.nasa.cmr.umm+json,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_418,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,12,False,application/vnd.nasa.cmr.umm+json,NASA_MAAP,ECHO_SYS,False,False,mmt_collection_426,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,30,False,application/vnd.nasa.cmr.umm+json,NASA_MAAP,ECHO_SYS,False,False,test_collection_feb23,False,False,...,NaN,N/A,1.0,"{""Description""=>""N/A""}",NaN,NaN,NaN,NaN,NaN,NaN
81,2,False,application/echo10+xml,ESA_MAAP,ECHO_SYS,False,False,TROPISAR,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The two tree branches, ['meta'] and ['umm'] in maap_data['items'] were converted in this dataframe. We can check the name of the columns below:

In [14]:
maap_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 79 columns):
 #   Column                                                                                                               Non-Null Count  Dtype  
---  ------                                                                                                               --------------  -----  
 0   meta.revision-id                                                                                                     83 non-null     int64  
 1   meta.deleted                                                                                                         83 non-null     bool   
 2   meta.format                                                                                                          83 non-null     object 
 3   meta.provider-id                                                                                                     83 non-null     object 
 4   meta.user-id    

<br>

Individual variables can be accessed if we know their indexes or field names. Like below:

<br>

In [26]:
(maap_normalized['umm.AdditionalAttributes'].iloc[0])[0]['Name']

'Dataset Status'

<br>
<br>

However, this is inefficient. What we need to do is to flatten the entire json fully so that the resulting dataframe contains:
1. items[0].umm.ShortName

2. items[0].umm.LocationKeywords[0], items[0].umm.LocationKeywords[1]

3. items[0].umm.Projects[0].ShortName, x.items[0].umm.Projects[0].LongName

4. items[0].umm.Platforms[0].ShortName, items[0].umm.Platforms[0].LongName,items[0].umm.Platforms[0].Instruments[0].ShortName, items[0].umm.Platforms[0].Instruments[0].LongName

5. items>umm>AdditionalAttributes> Spatial Resolution (2),DataFormat (4),AcquisitionType (8)

6. items[0].umm.ScienceKeywords[0].Category, items[0].umm.ScienceKeywords[0].Topic